## Einrichtung von Julia JuMP
---
Überprüfen Sie die bereits installierten Pakete. In der JuliaBox sind möglicherweise alle notwendigen Pakete bereits vorhanden.

In [1]:
using Pkg;
Pkg.status()

    Status `C:\Users\Kristian\.julia\environments\v1.2\Project.toml`
  [60bf3e95] GLPK v0.11.4
  [7073ff75] IJulia v1.20.0
  [4076af6c] JuMP v0.20.0
  [79b45036] Remark v0.1.0
  [f3b207a7] StatsPlots v0.12.0


Installieren Sie ``JuMP`` , mit dem Sie einfach Optimierungsprogramme formulieren können. Sowie ``GLPK`` einen open-source Solver zum Lösen der Problem und ``StatsPlots`` zur Visualisierung der Lösung.

In [2]:
#Pkg.add("JuMP")
#Pkg.add("GLPK");
#Pkg.add("StatsPlots");

Richten Sie die installierten Pakete so ein, dass sie im folgenden Code verwendet werden können.

In [3]:
using JuMP, GLPK, StatsPlots;

# Planspiel Schritt 2: Capacitated Lot Sizing Problem



### Ergebnisse aus Teil 1

In [4]:
F_Aggr = [0.0  0.0  0.0  0.0  0.0  0.0
         0.0  0.0  0.0  0.0  0.0  0.0
         0.0  0.0  0.0  0.0  0.0  0.0];

In [5]:
ZAggr = [21.0  6.0  0.0  0.0  38.0  40.0];

In [6]:
LAggr = [0.0  0.0  0.0  823.0  902.0  647.0  0.0
         0.0  0.0  0.0    0.0    1.0    0.0  0.0
         0.0  0.0  0.0    9.0    3.0    1.0  0.0 ];

## Erstellen Sie das Modell namens ``m`` und geben Sie als zu verwendenden Solver GLPK an.
---

In [7]:
m = Model(with_optimizer(GLPK.Optimizer));

### Mengen und Parameter

Fügen Sie die Mengen ein.


In [8]:
#Mengen
Produkte =   ["Lady", "Dude", "Rocker"];
Perioden =   [1,2,3,4];
Ressourcen = ["Herstellen"];


#Längen
I = length(Produkte);
T = length(Perioden);
J = length(Ressourcen);

Fügen Sie die Parameter ein.

In [9]:
#Hinweis: Überlegen Sie welche Parameter von Monats- auf Wochenwerte heruntergerechnet werden müssen

kl = [3.2/4, 4.32/4, 2.4/4];    #Lagerkostensatz der Produkte

ks = [50, 40, 25];              #Rüstkostensatz der Produkte

ts = [3, 1.5, 2];               #Rüstzeiten der Produkte

tp = [0.105, 0.095, 0.065];     #Stückbearbeitungszeiten der Produkte

M = 100000;                     #Große Zahl

                   
       #1,  2,  3,  4
d = [  350 350 350 350         #Lady
       300 300 300 300         #Dude
       250 250 250 250  ];     #Rocker 
#Bedarf von Produkt i in Periode t

kf = [320, 432, 240];        #Fremdbezugskostensatz 1 der Produkte (Tucher)

kv = [160, 216, 120];           #variable Herstellkosten der Produkte

In [10]:
#Diese Werte sind abhängig von der aggregierten Planung

l_start = [0,0,0];              #Anfangslagerbestand des Monats (Endbestand Vorperiode, s. oben)

l_end = [0,0,0];                #Der Endbestand des Monats ist immer der entsprechend berechnete Lagerbestand aus Schritt 1

c =  [80+6/4, 80+6/4, 80+6/4, 80+6/4];                        #Periodenkapazität
#Hinweis: Berücksichtigen Sie zusätzlich zur regulären Kapazität auch die in der aggr. Planung 
#         ermittelte Zusatzkapazität aufgeteilt auf 4 Wochen

faggr = [  0 0 0 0         #Lady
           0 0 0 0         #Dude
           0 0 0 0  ];     #Rocker ]             #Die wöchentliche Lieferung der bei Cheapress bestellten Produkte

### Entscheidungsvariablen
---
Definieren Sie die Entscheidungsvariablen. Achten Sie auf die Definitionsbereiche.

**Nichtnegativitätsbedingung**: Die Entscheidungsvariablen, also die Produktionsmengen, Lagerbestände, Fremdbezugsmengen und die in Anspruch genommenen Zusatzkapazitäten dürfen nur positive Werte annehmen, wir setzen deren Nichtnegativität voraus.

$ \qquad X_{it}, L_{it}, F_{it} \geq 0 \qquad \qquad \forall i \in I, t \in T$


In [11]:
@variables  m begin
            X[1:I,1:T]>=0,Int  #Produktionsmenge von Produkt i in Periode t
            F[1:I,1:T]>=0,Int  #Fremdbezugsmenge von Produkt i in Periode t
            L[1:I,0:T]>=0, Int; #Lagerbestand von Produkt i am Periodenende von t
            end

**Binaerbedingung**: Rüstvorgänge werden immer ganz ($\delta_{it} = 1$) oder gar nicht ($\delta_{it} = 0$) ausgeführt. Die binäre Rüstvariable nimmt also entweder den Wert 1 oder 0 an.

 $ \qquad  \gamma_{it} \in \{0,1\}  \qquad\qquad\qquad\qquad\qquad\qquad \forall t \in T, \quad i \in I $

In [12]:
@variable(m,gamma[1:I,1:T],Bin); #Die binäre Rüstvariable von Produkt i in Periode t

## Zielfunktion

**Kostenminimierung:** Der Produktionskosten K sollen minimiert werden. Diese berechnen sich hier aus der Summe der variablen Kosten, der Lagerkosten, der Fremdbezugskosten und Rüstkosten.

$$  \qquad \min K = \displaystyle\sum_{i=1}^I \sum_{t=1}^T (k_i^vX_{it}+k_i^l L_{it}+k_i^fF_{it}+ k^s_i\gamma_{i,t}) $$

In [13]:
@objective(m, Min,  sum(kv[i] * X[i,t] + kl[i] * L[i,t] + kf[i] * F[i,t] + ks[i]*gamma[i,t]
                    for i=1:I for t=1:T));

## Nebenbedingungen

**Kapazitaetsrestriktion:** Die Summe aus Produktions- und Rüstzeit aller Produkte darf in jeder Periode die vorhandene Kapazität der Ressource j nicht überschreiten.

 $$ \qquad \displaystyle\sum_{i=1}^I(t_i^p\cdot X_{it}+t_i^s\cdot\gamma_{it} ) \leq c_t \hspace{40mm} \forall t \in T, j \in J\qquad  $$ 
 


In [14]:
@constraint(m, KapRes[t=1:T, j=1:J], sum(ts[i] * gamma[i,t] + tp[i] * X[i,t] for i=1:I) <= c[t] );

**Lagerbilanzgleichung**: Der Lagerbestand eines Produktes am Ende einer Periode berechnet sich aus der Summe der eingelagerten Menge in der Vorperiode, der Produktionsmenge und der kurzfristigen und langfristigen Fremdbezugsmenge, abzüglich der abgesetzen Menge des Produktes.

$$ \qquad L_{it}=L_{i,t-1}+X_{it}-d_{it}+F_{it}+f_{it}^{aggr} \qquad \forall i \in I , t \in T$$

In [15]:
#Fügen sie den Parameter der langfristigen Fremdbezugsmenge in die Nebenbedingung ein!

@constraint(m, Lager[i=1:I,t=1:T], L[i,t] == L[i,t-1] + X[i,t] + F[i,t] - d[i,t]+faggr[i,t]);

**Anfangslagerbestand**: Der Anfangslagerbestand aller Produkte entspricht dem initial gesetzen $l_{start}$.

**Endlagerbestand**: Der Endlagerbestand aller Produkte entspricht dem initial gesetzen $l_{end}$.
 
  $$ \qquad L_{i,0} = l_{start} \hspace{40mm} \forall  i \in I$$
  
  $$ \qquad L_{i,T} = l_{end} \hspace{40mm} \forall  i \in I$$


In [16]:
@constraint(m, AnfLager[i=1:I], L[i,0] == l_start[i]);

#Fügen Sie eine Nebenbedingung für den Endlagerbestand hinzu

@constraint(m, EndLager[i=1:I], L[i,4] == l_end[i]);

**Rüstbedingung**: Wenn für ein Produkt in einer Periode nicht gerüstet wird, ist die produzierte Menge dieses Produkts in dieser Periode 0. Wenn für ein Produkt in einer Periode gerüstet wird, wird die produzierte Menge durch die Rüstbedingung nicht eingeschränkt.

 $$ \qquad X_{it} \leq M \cdot \gamma_{it} \hspace{40mm} \forall t \in T, \quad i \in I $$

In [17]:
@constraint(m, Ruestbed[i=1:I,t=1:T], X[i,t] <= M * gamma[i,t]);

In [ ]:
# Kann der Plan ohne Fremdbezug umgesetzt werden? 
# -> neue Nebenbedingung, die die Variable F = 0 setzt
# @constraint(m, KeinFremdbezug[i=1:I,t=1:T], F[i,t]==0);

## Lösen Sie das Modell.
---

In [18]:
optimize!(m)

Lassen Sie sich den Zielfunktionswert Z anzeigen.

In [19]:
db = JuMP.objective_value(m)

println("Objective value db: ", round(Int64,db))

Objective value db: 640518


Platz für weitere Berechnungen:

In [20]:
JuMP.value.(X)

3×4 Array{Float64,2}:
 133.0  461.0  289.0  289.0
 301.0  301.0  299.0  299.0
 499.0    0.0  250.0  250.0

In [21]:
JuMP.value.(F)

3×4 Array{Float64,2}:
 217.0  0.0  0.0  11.0
   0.0  0.0  0.0   0.0
   0.0  1.0  0.0   0.0

In [22]:
JuMP.value.(gamma)

3×4 Array{Float64,2}:
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  0.0  1.0  1.0

In [23]:
JuMP.value.(L)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:3
    Dimension 2, 0:4
And data, a 3×5 Array{Float64,2}:
 0.0    0.0  111.0  50.0  0.0
 0.0    1.0    2.0   1.0  0.0
 0.0  249.0    0.0   0.0  0.0

In [24]:
JuMP.value.(ts[i]*gamma[i,t] for i in 1:I, t in 1:T)

3×4 Array{Float64,2}:
 3.0  3.0  3.0  3.0
 1.5  1.5  1.5  1.5
 2.0  0.0  2.0  2.0

In [25]:
JuMP.value.(X[i,t]*tp[i] for i in 1:I, t in 1:T)

3×4 Array{Float64,2}:
 13.965  48.405  30.345  30.345
 28.595  28.595  28.405  28.405
 32.435   0.0    16.25   16.25 

In [26]:
JuMP.value.(sum(X[i,t]*kv[i] for i in 1:I, t in 1:T))

566600.0

In [27]:
JuMP.value.(sum(L[i,t]*kl[i] for i in 1:I, t in 1:T))

282.52000000000004

In [28]:
JuMP.value.(sum(ks[i]*gamma[i,t] for i in 1:I, t in 1:T))

435.0

In [29]:
JuMP.value.(sum(F[i,t]*kf[i] for i in 1:I, t in 1:T))

73200.0

In [30]:
JuMP.value.(sum(X[i,t] for i in 1:I, t in 1:T))

3371.0

In [31]:
JuMP.value.(sum(F[i,t] for i in 1:I, t in 1:T))

229.0

In [32]:
JuMP.value.(sum(gamma[1,t]*ts[1] for t in 1:T))

12.0

In [33]:
JuMP.value.(sum(gamma[2,t]*ts[2] for t in 1:T))

6.0

In [34]:
JuMP.value.(sum(gamma[3,t]*ts[3] for t in 1:T))

6.0